### Project 4: Evaluating Accuracy of Recommender Systems

In [2]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from scipy import linalg
from sklearn.model_selection import train_test_split
from numpy import nan
from sklearn.metrics import mean_squared_error as sk_rmse

In [4]:
import math as m
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
from surprise import SVD as f_SVD
from surprise import Dataset as Dt
from surprise import Reader
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise.model_selection import train_test_split as t_t_sp

In [19]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

### 1. The Dataset

In [7]:
path="C:/Users/talha/Documents/Training/CUNY Classes/Data612/Week1/ml-latest-small/"

In [8]:
ratings=pd.read_csv(path+'ratings.csv')

In [9]:
movie_names=pd.read_csv(path+'movies.csv')

#### 1.1 Extracting Movie Genres

In [42]:
movie_names.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [81]:
genres=movie_names['genres'].str.split('|',expand=True)

In [82]:
genres['ind']=movie_names.movieId

In [83]:
genres.head()

,0,1,2,3,4,5,6,7,8,9,ind
0,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None,1
1,Adventure,Children,Fantasy,None,None,None,None,None,None,None,2
2,Comedy,Romance,None,None,None,None,None,None,None,None,3
3,Comedy,Drama,Romance,None,None,None,None,None,None,None,4
4,Comedy,None,None,None,None,None,None,None,None,None,5


In [84]:
genres_m=genres.melt(id_vars='ind')

In [85]:
genres_m=genres_m[['ind','value']][~genres_m['value'].isnull()]

In [86]:
genres_m.head()

,ind,value
0,1,Adventure
1,2,Adventure
2,3,Comedy
3,4,Comedy
4,5,Comedy


In [87]:
genres_m[genres_m['ind']==1]

,ind,value
0,1,Adventure
9742,1,Animation
19484,1,Children
29226,1,Comedy
38968,1,Fantasy


In [88]:
lc=LabelEncoder()

In [89]:
ia=lc.fit_transform(genres_m['value'])

In [90]:
ia

array([ 2,  2,  5, ..., 14, 17, 19])

In [91]:
enc=OneHotEncoder(sparse=False)

In [92]:
ia = ia.reshape(len(ia), 1)
e_inc = enc.fit_transform(ia)

In [93]:
e_inc

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [94]:
categ_list=[]
for v in genres_m['value']:
    if v not in categ_list:
        categ_list.append(v)

In [95]:
e_inc=pd.DataFrame(e_inc, index=genres_m.index,columns=categ_list)

In [96]:
e_inc.head()

,Adventure,Comedy,Action,Drama,Crime,Children,Mystery,Animation,Documentary,Thriller,Horror,Fantasy,Western,Film-Noir,Romance,Sci-Fi,Musical,War,(no genres listed),IMAX
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
genres_m.head()

,ind,value
0,1,Adventure
1,2,Adventure
2,3,Comedy
3,4,Comedy
4,5,Comedy


In [98]:
genres_m=genres_m.merge(e_inc,left_index=True,right_index=True)

In [99]:
genres_m.head()

,ind,value,Adventure,Comedy,Action,Drama,Crime,Children,Mystery,Animation,...,Horror,Fantasy,Western,Film-Noir,Romance,Sci-Fi,Musical,War,(no genres listed),IMAX
0,1,Adventure,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Adventure,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Comedy,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Comedy,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Comedy,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
genres_m.sort_values(['ind'],ascending=True).head(n=10)

,ind,value,Adventure,Comedy,Action,Drama,Crime,Children,Mystery,Animation,...,Horror,Fantasy,Western,Film-Noir,Romance,Sci-Fi,Musical,War,(no genres listed),IMAX
0,1,Adventure,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19484,1,Children,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29226,1,Comedy,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38968,1,Fantasy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9742,1,Animation,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Adventure,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19485,2,Fantasy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9743,2,Children,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9744,3,Romance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,3,Comedy,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [101]:
genres_m.drop(['value'],axis=1,inplace=True)

In [102]:
genres_m=genres_m.groupby(['ind']).sum()

As result of the analysis above we have now converted all the genres classification as one-hot coded attributes for items which can be used as input to the similarity calculations. 

In [104]:
genres_m.tail()

,Adventure,Comedy,Action,Drama,Crime,Children,Mystery,Animation,Documentary,Thriller,Horror,Fantasy,Western,Film-Noir,Romance,Sci-Fi,Musical,War,(no genres listed),IMAX
ind,,,,,,,,,,,,,,,,,,,,
193581,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193587,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193609,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 1.2 Movie Ratings

In [105]:
#load dataset for the Surprise package
rat_t_surp=Dt.load_from_df(ratings[['userId', 'movieId', 'rating']], Reader(rating_scale=(1, 5)))

In [106]:
ratings_train,ratings_test=t_t_sp(rat_t_surp,test_size=0.3,random_state=324)

In [107]:
ratings.shape

(100836, 4)

In [109]:
trainset=rat_t_surp.build_full_trainset()

In [110]:
exist_test=trainset.build_anti_testset()

#### 1.3 Estimating the Ratings 

In [111]:
# We'll use the famous SVD algorithm.
algo = SVD(n_factors=5,n_epochs=10,verbose=True,biased=True,lr_all=0.005,reg_all=0.02)

In [112]:
algo.fit(ratings_train)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [113]:
#item based factors from SVD decomposition
algo.qi[0:3,]

array([[ 0.02685211,  0.06257886,  0.00113071,  0.08194054, -0.18210802],
       [ 0.17210148,  0.04135925,  0.17860486,  0.05536171,  0.12452713],
       [-0.07628916,  0.0206983 ,  0.12216703,  0.16604998, -0.06149495]])

In [458]:
#number of item factors
len(algo.qi)

8525

In [114]:
#user based factors from the SVD decomposition
algo.pu[0:3]

array([[-0.17199137, -0.09130149, -0.0257121 , -0.18538711,  0.07981888],
       [-0.07956545,  0.09559583, -0.05313257,  0.05296191,  0.07967689],
       [ 0.07246599,  0.07947109, -0.0140016 ,  0.03578512,  0.07142033]])

In [117]:
#calculation of the predictor from SVD
np.dot(algo.qi[0:4,],algo.pu[0:4].T)

array([[-0.04008732, -0.00638437, -0.00317072, -0.01506215],
       [-0.03829217, -0.0063753 ,  0.02413251, -0.06834052],
       [-0.02760186,  0.0054522 , -0.00404386, -0.01210042],
       [-0.01900841, -0.01145552, -0.01088932, -0.0323225 ]])

In [118]:
#item based biases - organized by inner id (see note on inner ids below)
algo.bi[0:4]

array([ 0.55255647,  0.12502098,  0.27048733, -0.03329931])

In [119]:
#user bias - organized by inner id
algo.bu[0:4]

array([ 0.19525248,  0.29798518, -0.11047053, -0.31504062])

In [120]:
ratings_train.global_mean

3.504236027484593

#### 1.4 Predicting the Ratings Using Surprise Package

In [121]:
#predict ratings for the test set
predictions=algo.test(ratings_test)

In [122]:
predictions=pd.DataFrame(predictions)

In [123]:
predictions.head()

,uid,iid,r_ui,est,details
0,606,40583,3.5,3.632181,{'was_impossible': False}
1,387,4034,3.0,3.647220,{'was_impossible': False}
2,571,3708,2.0,2.590285,{'was_impossible': False}
3,68,2405,3.5,2.856154,{'was_impossible': False}
4,390,8327,5.0,3.586192,{'was_impossible': False}


In [125]:
exist_test[0:4]

[(1, 318, 3.501556983616962),
 (1, 1704, 3.501556983616962),
 (1, 6874, 3.501556983616962),
 (1, 8798, 3.501556983616962)]

In [126]:
#the missing ratings in the training set
missing=ratings_train.build_anti_testset()

In [127]:
#predict the missing ratings and return a dataframe
missing_pred=algo.test(missing)
missing_pred=pd.DataFrame(missing_pred)

In [128]:
#the head of the missing predictions - includes both test set and ratings for 
missing_pred.head()

,uid,iid,r_ui,est,details
0,302,2117,3.504236,3.786217,{'was_impossible': False}
1,302,122882,3.504236,3.942374,{'was_impossible': False}
2,302,4915,3.504236,3.647181,{'was_impossible': False}
3,302,67734,3.504236,3.764277,{'was_impossible': False}
4,302,104218,3.504236,3.700072,{'was_impossible': False}


In [129]:
#extract the training data for reference and further analysis
tr=[i for i in ratings_train.all_ratings()]

In [130]:
tr[0:5]

[(0, 0, 4.0), (0, 671, 3.0), (0, 1044, 3.0), (0, 116, 5.0), (0, 15, 3.0)]

In [131]:
#item inner-ids for the training set
it_id=[idk for ud,idk,r in ratings_train.all_ratings()]

In [132]:
it_id[0:10]

[0, 671, 1044, 116, 15, 466, 554, 659, 762, 896]

In [133]:
#total number of ratings in the training set
len(it_id)

70585

In [134]:
#total number of users in the training set
len(algo.pu)

610

In [135]:
tr_df=pd.DataFrame(tr, columns=['uiid','iid','rating'])

In [136]:
# the surprise package creates inner ids (a set of alternate ids) to track users and items; 
#data set item and user ids are referred to as raw ids; the functions below take inner-ids as inputs and return raw-ids 
#which can be used to map other attributes like movie names or movie genres for example. 
def get_uid(uid,trainset):
    """
    args: 
    uid: the inner id of the user
    returns:
    returns the raw id of the user
    """
    try:
        return trainset.to_raw_uid(uid)
    except ValueError: # user was not part of the trainset
        return "Error"

def get_iid(iid,trainset):
    """
    args:
    iid: the raw id of the item
    returns:
    the number of users that have rated the item.
    """
    try:
        return trainset.to_raw_iid(iid)
    except ValueError:
        return "Error"

In [137]:
uid_list=[get_uid(i,ratings_train) for i in tr_df.uiid] #list of raw user ids
iid_list=[get_iid(i,ratings_train) for i in tr_df.iid] #list of raw item ids

In [138]:
tr_df['raw_uid']=uid_list
tr_df['raw_iid']=iid_list

In [139]:
tr_df.head()

,uiid,iid,rating,raw_uid,raw_iid
0,0,0,4.0,302,457
1,0,671,3.0,302,293
2,0,1044,3.0,302,736
3,0,116,5.0,302,1221
4,0,15,3.0,302,628


In [140]:
def return_id(id_type): 
    #id_type: Str of "user" or "item"
    #the function returns internal ids of the ratings
    if id_type=='item':
        it_id=[idk for ud,idk,r in ratings_train.all_ratings()]
    elif id_type=='user':
        it_id=[ud for ud,idk,r in ratings_train.all_ratings()]
    else: 
        return print("must be 'user' or 'item'")
    unique_id=[]
    for i in it_id:
        if i not in unique_id:
            unique_id.append(i)
    return unique_id

In [141]:
#setup the item-factors as a dataframe
item_factors=pd.DataFrame(algo.qi)

In [142]:
unique_id=[] # returns the set of collapsed inner ids in the training set
for i in tr_df.iid:
    if i not in unique_id:
        unique_id.append(i)

In [146]:
unique_id=sorted(unique_id)

In [147]:
item_factors['item_id']=unique_id

In [189]:
unique_uid=[] # returns the set of collapsed inner ids in the training set
for i in tr_df.uiid:
    if i not in unique_uid:
        unique_uid.append(i)

In [190]:
unique_uid=sorted(unique_uid)

In [148]:
iid_list=[get_iid(i,ratings_train) for i in item_factors.item_id] #list of raw item ids

In [149]:
item_factors['raw_id']=iid_list #add raw ids to the item factors

In [150]:
item_factors.head()

,0,1,2,3,4,item_id,raw_id
0,0.026852,0.062579,0.001131,0.081941,-0.182108,0,457
1,0.172101,0.041359,0.178605,0.055362,0.124527,1,2117
2,-0.076289,0.020698,0.122167,0.166050,-0.061495,2,122882
3,0.009213,-0.159759,-0.035521,0.149263,-0.065800,3,4915
4,0.084405,0.178154,-0.060512,-0.073843,0.176620,4,67734


In [154]:
#merge the item factors with the movie genre attributes
item_factors_m=item_factors.merge(genres_m,left_on='raw_id',right_index=True)

In [155]:
item_factors_m=item_factors_m.set_index('raw_id')

In [197]:
#merge item biases and user biases into dataframes
item_df=pd.DataFrame(zip(unique_id,algo.bi),columns=['id','item_bias'])

In [198]:
#merge item biases and user biases into dataframes
user_df=pd.DataFrame(zip(unique_uid,algo.bu),columns=['id','user_bias'])

In [199]:
tr_df.head()

,uiid,iid,rating,raw_uid,raw_iid
0,0,0,4.0,302,457
1,0,671,3.0,302,293
2,0,1044,3.0,302,736
3,0,116,5.0,302,1221
4,0,15,3.0,302,628


In [200]:
#merge into a single dataframe
tr_df=tr_df.merge(user_df,left_on='uiid',right_on='id')

In [201]:
#merge into a single dataframe
tr_df=tr_df.merge(item_df,left_on='iid',right_on='id')

In [203]:
tr_df.drop(columns=['id_x','id_y'],inplace=True)

In [204]:
tr_df['mean']=ratings_train.global_mean

In [205]:
tr_df['baseline']=tr_df['mean']+tr_df['user_bias']+tr_df['item_bias']

In [206]:
tr_df.head()

,uiid,iid,rating,raw_uid,raw_iid,user_bias,item_bias,mean,baseline
0,0,0,4.0,302,457,0.195252,0.552556,3.504236,4.252045
1,6,0,3.0,288,457,-0.410322,0.552556,3.504236,3.646470
2,7,0,5.0,474,457,-0.141356,0.552556,3.504236,3.915437
3,10,0,4.0,584,457,0.726690,0.552556,3.504236,4.783483
4,11,0,3.5,18,457,0.060436,0.552556,3.504236,4.117229


In [207]:
tr_df['pred']=tr_df['rating']-tr_df['baseline']

#### 1.5 Define Functions to calculate similarities 

In [306]:
item_factors_m[[0,1,2,3,4]].head()

,0,1,2,3,4
raw_id,,,,,
457,0.026852,0.062579,0.001131,0.081941,-0.182108
2117,0.172101,0.041359,0.178605,0.055362,0.124527
122882,-0.076289,0.020698,0.122167,0.166050,-0.061495
4915,0.009213,-0.159759,-0.035521,0.149263,-0.065800
67734,0.084405,0.178154,-0.060512,-0.073843,0.176620


In [318]:
item_factors_m.drop(columns=['item_id'],inplace=True)

In [319]:
item_factors_m.head()

,0,1,2,3,4,Adventure,Comedy,Action,Drama,Crime,...,Horror,Fantasy,Western,Film-Noir,Romance,Sci-Fi,Musical,War,(no genres listed),IMAX
raw_id,,,,,,,,,,,,,,,,,,,,,
457,0.026852,0.062579,0.001131,0.081941,-0.182108,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2117,0.172101,0.041359,0.178605,0.055362,0.124527,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
122882,-0.076289,0.020698,0.122167,0.166050,-0.061495,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4915,0.009213,-0.159759,-0.035521,0.149263,-0.065800,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67734,0.084405,0.178154,-0.060512,-0.073843,0.176620,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [324]:
#define function to find the Neighborhood of user / item
#find the similarity of each user / item, and identify top k users / items
#return the neighborhood of users / items
def Neighbor_sim(a,k): # a is a matrix of user and items, and neighborhood of k
    items=np.arange(len(a))
    sim=cosine_similarity(a,a)
    ia=list(a.index)
    Nlist=[]
    Slist=[]
    k=k+1 #add one since the first includes the user himself
    for i in items:
        print("processing item...",i)
        ib=list(sim[i])
        z=[ia for ib,ia in sorted(zip(ib,ia),reverse=True)]
        Slist.append(sorted(sim[i],reverse=True)[0:k])
        Nlist.append(z[0:k])
    Slist=pd.DataFrame(Slist)
    Slist['index']=ia
    Slist=Slist.set_index('index')
    Slist.drop([0],axis=1,inplace=True)
    Nlist=pd.DataFrame(Nlist)
    Nlist=Nlist.set_index(0)
    print('similarity calculations completed...')
    return (Nlist,Slist)

In [228]:
b.index

Int64Index([457, 2117, 122882, 4915, 67734], dtype='int64', name='index')

In [224]:
tr_df.head()

,uiid,iid,rating,raw_uid,raw_iid,user_bias,item_bias,mean,baseline,pred
0,0,0,4.0,302,457,0.195252,0.552556,3.504236,4.252045,-0.252045
1,6,0,3.0,288,457,-0.410322,0.552556,3.504236,3.646470,-0.646470
2,7,0,5.0,474,457,-0.141356,0.552556,3.504236,3.915437,1.084563
3,10,0,4.0,584,457,0.726690,0.552556,3.504236,4.783483,-0.783483
4,11,0,3.5,18,457,0.060436,0.552556,3.504236,4.117229,-0.617229


In [275]:
#define a weighting average function for k users
#rj is a set of ratings for item j
#sa are the similarities for users / items. 
def weight_k(sa,rj,k):
    return(np.dot(sa[0:k],rj[0:k])/sum(sa[0:k]))

In [312]:
#for user i, and given a set of movies, predict highest rated / recommended movies for user
def item_collab(usr,it,item_mat,factors,k):
#user is a list of users for whom we want to predict ratings 
#it is a list of items / movies
#user_mat is a dense matrix, as a dataframe, of users (as rows) and items (across columns)
#factors are the reduced form factors (gotten from Funk SVD) over which similarity is to be calculated 
#k is a parameter for the size of the neighborhood 
    usr=list(usr)
    it=list(it)
    itemid,sims=Neighbor_sim(factors,k) #calculate similarities and neighborhoods of all movies
    predict=[]
    for i in it: # for each movie / item
        print("processing item ...",i)
        for e in usr: # for each user 
            item_mat_s=item_mat.filter(items=list([e]),axis=1) #filter for movies of interest
            clist_i=itemid.loc[i,:] #return the most similar movies for user e
            item_mat_s_r=item_mat_s.loc[clist_i] #filter matrix, i.e. row filter
            item_mat_s_r=item_mat_s_r.squeeze()
            sims_s=sims.loc[i,:]
            rt=weight_k(sims_s,item_mat_s_r,k)
            predict.append([e,i,rt])
    predict=pd.DataFrame(predict,columns=['userid','item','item_pred'])
    return (predict)

In [305]:
#for user i, and given a set of movies, predict highest rated / recommended movies for user
def user_collab(usr,it,usr_mat,k):
#user is a list of users for whom we want to predict ratings 
#it is a list of items / movies
#user_mat is a dense matrix, as a dataframe, of users (as rows) and items (across columns)
#k is a parameter for the size of the neighborhood 
    usr=list(usr)
    it=list(it)
    usrid,sims=Neighbor_sim(usr_mat,k) #calculate similarities and neighborhoods of all users
    predict=[]
    for e in usr: # for each user, need to exclude select user and focus on similar user
        #filt_list=[all_usr for all_usr in all_usr if all_usr!=e] #filter the list so that current user is excluded
        for i in it: #for each movie / item
            usr_mat_s=usr_mat.filter(items=list([i]),axis=1) #filter for movies of interest
            clist_u=usrid.loc[e,:] #return the most similar users for user e
            usr_mat_s_r=usr_mat_s.loc[clist_u] #filter matrix, i.e. row filter
            usr_mat_s_r=usr_mat_s_r.squeeze()
            sims_s=sims.loc[e,:]
            rt=weight_k(sims_s,usr_mat_s_r,k)
            predict.append([e,i,rt])
            #need to calculate the weights here
    predict=pd.DataFrame(predict,columns=['userid','item','user_pred'])
    return (predict)

#### 1.6 Predicting the Ratings Using Surprise Package SVD Factors and Similarity Functions 

In [307]:
#reshape the data
usr_mat=tr_df[['raw_uid','raw_iid','pred']].pivot(index='raw_uid', columns='raw_iid',values='pred')

In [308]:
#replace missing ratings with 
usr_mat=usr_mat.replace(nan,0)
usr_mat.head()

raw_iid,1,2,3,4,5,6,7,8,9,10,...,190209,190213,190219,193565,193567,193579,193583,193585,193587,193609
raw_uid,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.0,-0.098961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.054599,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [309]:
usr_mat.tail()

raw_iid,1,2,3,4,5,6,7,8,9,10,...,190209,190213,190219,193565,193567,193579,193583,193585,193587,193609
raw_uid,,,,,,,,,,,,,,,,,,,,,
606,-1.508944,0.000000,0.0,0.0,0.0,0.000000,-0.761613,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,-0.219195,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,-1.059586,-1.128325,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.991547,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
609,-0.725168,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
610,0.000000,0.000000,0.0,0.0,0.0,0.914658,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [314]:
#calculate the results using SVD factors for similarity and baseline ratings
item_collab(list(usr_mat.index),list(item_factors_m[[0,1,2,3,4]].index),usr_mat.T,item_factors_m[[0,1,2,3,4]],3)

KeyboardInterrupt: 

In [ ]:
#calculate the results using SVD factors for similarity and baseline ratings
res_all=item_collab(list(usr_mat.index),list(item_factors_m.index),usr_mat.T,item_factors_m,3)

processing item... 0
processing item... 1
processing item... 2
processing item... 3
processing item... 4
processing item... 5
processing item... 6
processing item... 7
processing item... 8
processing item... 9
processing item... 10
processing item... 11
processing item... 12
processing item... 13
processing item... 14
processing item... 15
processing item... 16
processing item... 17
processing item... 18
processing item... 19
processing item... 20
processing item... 21
processing item... 22
processing item... 23
processing item... 24
processing item... 25
processing item... 26
processing item... 27
processing item... 28
processing item... 29
processing item... 30
processing item... 31
processing item... 32
processing item... 33
processing item... 34
processing item... 35
processing item... 36
processing item... 37
processing item... 38
processing item... 39
processing item... 40
processing item... 41
processing item... 42
processing item... 43
processing item... 44
processing item... 4

processing item... 366
processing item... 367
processing item... 368
processing item... 369
processing item... 370
processing item... 371
processing item... 372
processing item... 373
processing item... 374
processing item... 375
processing item... 376
processing item... 377
processing item... 378
processing item... 379
processing item... 380
processing item... 381
processing item... 382
processing item... 383
processing item... 384
processing item... 385
processing item... 386
processing item... 387
processing item... 388
processing item... 389
processing item... 390
processing item... 391
processing item... 392
processing item... 393
processing item... 394
processing item... 395
processing item... 396
processing item... 397
processing item... 398
processing item... 399
processing item... 400
processing item... 401
processing item... 402
processing item... 403
processing item... 404
processing item... 405
processing item... 406
processing item... 407
processing item... 408
processing 

processing item... 725
processing item... 726
processing item... 727
processing item... 728
processing item... 729
processing item... 730
processing item... 731
processing item... 732
processing item... 733
processing item... 734
processing item... 735
processing item... 736
processing item... 737
processing item... 738
processing item... 739
processing item... 740
processing item... 741
processing item... 742
processing item... 743
processing item... 744
processing item... 745
processing item... 746
processing item... 747
processing item... 748
processing item... 749
processing item... 750
processing item... 751
processing item... 752
processing item... 753
processing item... 754
processing item... 755
processing item... 756
processing item... 757
processing item... 758
processing item... 759
processing item... 760
processing item... 761
processing item... 762
processing item... 763
processing item... 764
processing item... 765
processing item... 766
processing item... 767
processing 

processing item... 1080
processing item... 1081
processing item... 1082
processing item... 1083
processing item... 1084
processing item... 1085
processing item... 1086
processing item... 1087
processing item... 1088
processing item... 1089
processing item... 1090
processing item... 1091
processing item... 1092
processing item... 1093
processing item... 1094
processing item... 1095
processing item... 1096
processing item... 1097
processing item... 1098
processing item... 1099
processing item... 1100
processing item... 1101
processing item... 1102
processing item... 1103
processing item... 1104
processing item... 1105
processing item... 1106
processing item... 1107
processing item... 1108
processing item... 1109
processing item... 1110
processing item... 1111
processing item... 1112
processing item... 1113
processing item... 1114
processing item... 1115
processing item... 1116
processing item... 1117
processing item... 1118
processing item... 1119
processing item... 1120
processing item.

processing item... 1427
processing item... 1428
processing item... 1429
processing item... 1430
processing item... 1431
processing item... 1432
processing item... 1433
processing item... 1434
processing item... 1435
processing item... 1436
processing item... 1437
processing item... 1438
processing item... 1439
processing item... 1440
processing item... 1441
processing item... 1442
processing item... 1443
processing item... 1444
processing item... 1445
processing item... 1446
processing item... 1447
processing item... 1448
processing item... 1449
processing item... 1450
processing item... 1451
processing item... 1452
processing item... 1453
processing item... 1454
processing item... 1455
processing item... 1456
processing item... 1457
processing item... 1458
processing item... 1459
processing item... 1460
processing item... 1461
processing item... 1462
processing item... 1463
processing item... 1464
processing item... 1465
processing item... 1466
processing item... 1467
processing item.

...

processing item ... 1046
processing item ... 5884
processing item ... 38798
processing item ... 5782
processing item ... 3000
processing item ... 45672
processing item ... 8869
processing item ... 136666
processing item ... 6686
processing item ... 575
processing item ... 143355
processing item ... 3709
processing item ... 3087
processing item ... 3702
processing item ... 799
processing item ... 2826
processing item ... 90531
processing item ... 53953
processing item ... 8376
processing item ... 50923
processing item ... 2378
processing item ... 85
processing item ... 32009
processing item ... 2291
processing item ... 6979
processing item ... 3299
processing item ... 140110
processing item ... 50189
processing item ... 160080
processing item ... 27822
processing item ... 76251
processing item ... 4211
processing item ... 2094
processing item ... 2312
processing item ... 48982
processing item ... 8783
processing item ... 111844
processing item ... 3703
processing item ... 36519
processi

processing item ... 104841
processing item ... 60040
processing item ... 81782
processing item ... 4017
processing item ... 108190
processing item ... 2551
processing item ... 1245
processing item ... 3948
processing item ... 852
processing item ... 552
processing item ... 942
processing item ... 158
processing item ... 1271
processing item ... 6909
processing item ... 110127
processing item ... 5630
processing item ... 7327
processing item ... 30810
processing item ... 64614
processing item ... 115569
processing item ... 550
processing item ... 47261
processing item ... 2142
processing item ... 180297
processing item ... 6336
processing item ... 509
processing item ... 114180
processing item ... 1862
processing item ... 5218
processing item ... 75416
processing item ... 5444
processing item ... 5619
processing item ... 3654
processing item ... 3959
processing item ... 1373
processing item ... 471
processing item ... 43560
processing item ... 159755
processing item ... 64839
processing

processing item ... 1272
processing item ... 7360
processing item ... 303
processing item ... 5633
processing item ... 1282
processing item ... 65188
processing item ... 50872
processing item ... 2159
processing item ... 128620
processing item ... 53464
processing item ... 23
processing item ... 3185
processing item ... 4519
processing item ... 319
processing item ... 100083
processing item ... 3814
processing item ... 53894
processing item ... 3213
processing item ... 1027
processing item ... 1375
processing item ... 7079
processing item ... 2496
processing item ... 50794
processing item ... 2266
processing item ... 26554
processing item ... 1223
processing item ... 1269
processing item ... 4052
processing item ... 3298
processing item ... 1590
processing item ... 5784
processing item ... 81910
processing item ... 52458
processing item ... 724
processing item ... 190207
processing item ... 5324
processing item ... 127198
processing item ... 122922
processing item ... 243
processing it

processing item ... 779
processing item ... 5225
processing item ... 6810
processing item ... 77709
processing item ... 761
processing item ... 204
processing item ... 1855
processing item ... 75
processing item ... 1050
processing item ... 6617
processing item ... 5066
processing item ... 209
processing item ... 1031
processing item ... 4794
processing item ... 5266
processing item ... 46
processing item ... 60756
processing item ... 2370
processing item ... 1951
processing item ... 2478
processing item ... 5346
processing item ... 51080
processing item ... 2424
processing item ... 5013
processing item ... 98154
processing item ... 2206
processing item ... 61
processing item ... 6755
processing item ... 459
processing item ... 155288
processing item ... 163937
processing item ... 1960
processing item ... 6051
processing item ... 2048
processing item ... 728
processing item ... 140174
processing item ... 798
processing item ... 27592
processing item ... 4241
processing item ... 3173
pr

processing item ... 97024
processing item ... 4649
processing item ... 169984
processing item ... 348
processing item ... 5093
processing item ... 1694
processing item ... 4932
processing item ... 3825
processing item ... 4855
processing item ... 1327
processing item ... 5943
processing item ... 25937
processing item ... 188
processing item ... 56176
processing item ... 49286
processing item ... 25963
processing item ... 4671
processing item ... 4992
processing item ... 3066
processing item ... 8
processing item ... 87306
processing item ... 81788
processing item ... 100498
processing item ... 2950
processing item ... 67508
processing item ... 2732
processing item ... 6658
processing item ... 55768
processing item ... 82
processing item ... 7373
processing item ... 6213
processing item ... 4840
processing item ... 117529
processing item ... 8507
processing item ... 108981
processing item ... 6193
processing item ... 2110
processing item ... 69805
processing item ... 93840
processing it

#### 1.7 Calculate Training Error for Surprise package

In [424]:
tr_tuple=list(map(tuple,tr_df[['raw_uid','raw_iid','rating']].to_numpy()))

In [425]:
tr_tuple[0:3]

[(302.0, 457.0, 4.0), (302.0, 293.0, 3.0), (302.0, 736.0, 3.0)]

In [432]:
train_pred=algo.test(tr_tuple)
train_pred=pd.DataFrame(train_pred)

In [436]:
train_pred.head()

,uid,iid,r_ui,est,details
0,302.0,457.0,4.0,4.290220,{'was_impossible': False}
1,302.0,293.0,3.0,4.271103,{'was_impossible': False}
2,302.0,736.0,3.0,3.665508,{'was_impossible': False}
3,302.0,1221.0,5.0,4.407071,{'was_impossible': False}
4,302.0,628.0,3.0,3.684504,{'was_impossible': False}


In [437]:
sk_rmse(train_pred[['r_ui']],train_pred[['est']])

0.7108660427181629

In [579]:
missing_pred_df.head()

,uid,iid,r_ui,est,details
0,1,318,3.501557,5.000000,{'was_impossible': False}
1,1,1704,3.501557,4.862736,{'was_impossible': False}
2,1,6874,3.501557,4.756050,{'was_impossible': False}
3,1,8798,3.501557,4.467686,{'was_impossible': False}
4,1,46970,3.501557,4.188977,{'was_impossible': False}


In [386]:
ratings_train.to_raw_uid(0)

302